In [2]:
import glob
dataset = "CARLA"
exp = "02-11_07-00_vgg_NoKPI_projhead_con05_io10_mask1_CARLA_5e-05_0.0001"
device = ""
gpu = 3

if device == "TWCC":
    gpu = 0

    root = "/home/leohuang0511"
else:
    root = "/nfs/home/leo0511/Research"
files = sorted(glob.glob(f"{root}/SSSP/exp/{dataset}/FT/{exp}/ep*"))
for file in files:
    print(f"python test_{dataset}.py --MODEL_PATH {file} --GPU_ID {gpu}")

python test_CARLA.py --MODEL_PATH /nfs/home/leo0511/Research/SSSP/exp/CARLA/FT/02-11_07-00_vgg_NoKPI_projhead_con05_io10_mask1_CARLA_5e-05_0.0001/ep_10_iter_16000_mae_28.865_mse_29.849_seq_MAE_23.532_seq_MSE_23.532_WRAE_10.321_MIAE_2.174_MOAE_1.935.pth --GPU_ID 3
python test_CARLA.py --MODEL_PATH /nfs/home/leo0511/Research/SSSP/exp/CARLA/FT/02-11_07-00_vgg_NoKPI_projhead_con05_io10_mask1_CARLA_5e-05_0.0001/ep_11_iter_18000_mae_28.037_mse_28.924_seq_MAE_41.489_seq_MSE_41.489_WRAE_18.197_MIAE_1.638_MOAE_2.445.pth --GPU_ID 3
python test_CARLA.py --MODEL_PATH /nfs/home/leo0511/Research/SSSP/exp/CARLA/FT/02-11_07-00_vgg_NoKPI_projhead_con05_io10_mask1_CARLA_5e-05_0.0001/ep_1_iter_1000_mae_7.547_mse_8.583_seq_MAE_304.827_seq_MSE_304.827_WRAE_133.696_MIAE_19.535_MOAE_18.179.pth --GPU_ID 3
python test_CARLA.py --MODEL_PATH /nfs/home/leo0511/Research/SSSP/exp/CARLA/FT/02-11_07-00_vgg_NoKPI_projhead_con05_io10_mask1_CARLA_5e-05_0.0001/ep_2_iter_2000_mae_13.311_mse_14.139_seq_MAE_297.245_seq_MSE_

In [1]:
import torch
import argparse
from importlib import import_module


parser = argparse.ArgumentParser()
parser.add_argument('--EXP_NAME', type=str, default='')

parser.add_argument('--RESUME_PATH',type=str, default='')
parser.add_argument('--PRETRAIN_PATH',type=str, default='')
parser.add_argument('--FROZEN', default=False, action='store_true', help="frozen pretrained frontend weights")


parser.add_argument('--GPU_ID', type=str, default='0')
parser.add_argument('--SEED', type=int, default=3035)
parser.add_argument('--DATASET', type=str, default='HT21')
parser.add_argument('--TASK', type=str, default='FT')
parser.add_argument('--PRINT_FREQ', type=int, default=20)
parser.add_argument('--SAVE_VIS_FREQ', type=int, default=500)
parser.add_argument('--BACKBONE', type=str, default='vgg')


parser.add_argument('--LR_MIN', type=float, default=1e-6)
parser.add_argument('--LR_BASE', type=float, default=5e-5, help='density branch')
parser.add_argument('--LR_THRE', type=float, default=1e-4, help='mask branch')
parser.add_argument('--LR_DECAY', type=float, default=0.95)
parser.add_argument('--WEIGHT_DECAY', type=float, default=1e-5)
parser.add_argument('--WARMUP_EPOCH', type=int, default=3, help='number of epochs for warm up step in cosine annealing lr scheduler')
parser.add_argument('--MAX_EPOCH', type=int, default=20)
parser.add_argument('--WORKER', type=int, default=4)


parser.add_argument('--CON_WEIGHT', type=float, default=0.5)
parser.add_argument('--SCALE_WEIGHT', type=float, nargs='+', default=[2,0.1,0.01])
parser.add_argument('--CNT_WEIGHT', type=float, default=10)
parser.add_argument('--MASK_WEIGHT', type=float, default=1)
parser.add_argument('--IO_WEIGHT', type=float, default=1)




#_test or val
parser.add_argument('--VAL_FREQ', type=int, default=1000)
parser.add_argument('--VAL_START', type=int, default=1)
parser.add_argument('--VAL_BATCH_SIZE', type=int, default=1)



#_train
parser.add_argument('--TRAIN_SIZE', type=int, nargs='+', default=[768,1024])
parser.add_argument('--TRAIN_FRAME_INTERVALS', type=int, nargs='+', default=[40, 85])
parser.add_argument('--TRAIN_BATCH_SIZE', type=int, default=2)
parser.add_argument('--ROI_RADIUS', type=float, default=4.)
parser.add_argument('--FEATURE_SCALE', type=float, default=1/4.)
parser.add_argument('--GAUSSIAN_SIGMA', type=float, default=4)
parser.add_argument('--CONF_BLOCK_SIZE', type=int, default=16)




#_shift pretrain
parser.add_argument('--WIN_OFFSET_RANGE', type=int, nargs='+', default=[100,350])
parser.add_argument('--IMG_OFFSET_RANGE', type=int, nargs='+', default=[-100,100])


parser.add_argument('--DEN_FACTOR', type=float, default=200.)
parser.add_argument('--MEAN_STD', type=tuple, default=([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]))

# cfg = parser.parse_args()
cfg = parser.parse_known_args()[0]


# data_mode = cfg.DATASET
# datasetting = import_module(f'datasets.setting.{data_mode}')
# cfg_data = datasetting.cfg_data
cfg_data = cfg



In [10]:
import sys
sys.path.append('..')
from model.SMDCA import  SMDCANet

model = SMDCANet(cfg, cfg_data).Extractor.scale_loc_head
total = sum([param.nelement() for param in model.parameters()])
print("Number of parameter: %.2fM" % (total/1e6))

Number of parameter: 0.16M


In [5]:
import sys
sys.path.append('../../../')

from FMDC.model.video_crowd_count import  video_crowd_count
sys.path.append('./FMDC')


model = video_crowd_count(cfg, cfg_data)
total = sum([param.nelement() for param in model.parameters()])
print("Number of parameter: %.2fM" % (total/1e6))

ModuleNotFoundError: No module named 'misc'

In [8]:
import thop
from thop import profile 
input = torch.randn(4, 3, 768, 1024)
flops, params = profile(model, inputs=(input,))
print(flops)
print(params)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.ConvTranspose2d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.instancenorm.InstanceNorm2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.


TypeError: video_crowd_count.forward() missing 2 required positional arguments: 'img_rgb' and 'target'